``` 
.\llama-server.exe -m "C:\Users\jean\Documents\Llama\Models\bartowski_Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf" --port 8080 -v --verbose-prompt --jinja
```

In [1]:
import openai

In [2]:
client = openai.OpenAI(
    base_url="http://localhost:8080/v1", 
    api_key = "sk-no-key-required"
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "What is the weather today in Guanajuato?"},
  ]
)

In [3]:
print(completion.__verbose['prompt'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 22 Oct 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the weather today in Guanajuato?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [4]:
print(completion.choices[0].message.content)

I'm not able to access current weather conditions, but I can suggest some options to find out the current weather in Guanajuato. 

1. Check a weather website: You can check websites like AccuWeather, Weather.com, or the National Meteorological Service website for Mexico (Servicio Meteorológico Nacional) to get the current weather conditions in Guanajuato.

2. Use a search engine: You can type "weather in Guanajuato" in a search engine like Google to find the current weather conditions.

3. Check social media: Follow local news or weather accounts on social media platforms like Twitter or Facebook to get updates on the current weather in Guanajuato.

Please note that the weather can change quickly, and it's always a good idea to check multiple sources for the most accurate information.


#### Completions

In [5]:
client = openai.OpenAI(
    base_url="http://localhost:8080/v1", 
    api_key = "sk-no-key-required"
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "Hi, how are you?"},
    {"role": "assistant", "content": "I am doing well, thank you! How can I assist you today?"},
    {"role": "user", "content": "Can you tell me a joke?"},
  ]
)

In [6]:
print(completion.__verbose['prompt'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 22 Oct 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Hi, how are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I am doing well, thank you! How can I assist you today?<|eot_id|><|start_header_id|>user<|end_header_id|>

Can you tell me a joke?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [7]:
print(completion.choices[0].message.content)

A man walked into a library and asked the librarian, "Do you have any books on Pavlov's dogs and Schrödinger's cat?" 

The librarian replied, "It rings a bell, but I'm not sure if it's here or not."


#### Function calling

In [8]:
get_weather_schema =  {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ]
        }
    }
}

The actual function will be used for calling

In [9]:
def get_weather(location):
    print(f"Getting weather for {location}...")
    # Simulate a weather API call
    return f"The current temperature in {location} is 25°C."

In [10]:
chat_history = [
        #{"role": "system", "content": "You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal user question."},
        {"role": "user", "content": "Hi, what is the weather today in Bogota?"}
    ]

In [11]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=chat_history,
    tools=[get_weather_schema]
)

In [12]:
print(completion.__verbose['prompt'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 22 Oct 2025

You have access to the following functions. To call a function, please respond with JSON for a function call.Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.Do not use variables.

{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ]
        }
    }
}

<|eot_id|><|start_header_id|>user<|end_header_id|>

Hi, what is the weather today in Bogota?<|eot_id|><|start_header_id|>assista

In [13]:
print(completion.choices[0].message.content)

None


In [14]:
print(completion.choices[0].message.tool_calls[0].function)

Function(arguments='{"location":"Bogota, Colombia"}', name='get_weather')


In [15]:
import ast
if completion.choices[0].message.tool_calls:
    if completion.choices[0].message.tool_calls[0].function.name == 'get_weather':
        response = get_weather(ast.literal_eval(completion.choices[0].message.tool_calls[0].function.arguments)['location'])
        print(response)

Getting weather for Bogota, Colombia...
The current temperature in Bogota, Colombia is 25°C.


In [16]:
response = str({"output": response}) # If I don't return it in form of this dict. The model doesn't responds correctly. It's in the Llama documentation also https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_1/

In [17]:
response

"{'output': 'The current temperature in Bogota, Colombia is 25°C.'}"

In [18]:
chat_history.append({"role": "assistant", "tool_calls": completion.choices[0].message.tool_calls})
chat_history.append({"role": "tool", "content": response})

In [19]:
chat_history

[{'role': 'user', 'content': 'Hi, what is the weather today in Bogota?'},
 {'role': 'assistant',
  'tool_calls': [ChatCompletionMessageToolCall(id='ryqlnqKrQmfGo9kNYQyGswrWa6xlCbT9', function=Function(arguments='{"location":"Bogota, Colombia"}', name='get_weather'), type='function')]},
 {'role': 'tool',
  'content': "{'output': 'The current temperature in Bogota, Colombia is 25°C.'}"}]

In [20]:
chat_history = [{"role": "system", "content": "You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal user question."}] + chat_history

In [21]:
chat_history

[{'role': 'system',
  'content': 'You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal user question.'},
 {'role': 'user', 'content': 'Hi, what is the weather today in Bogota?'},
 {'role': 'assistant',
  'tool_calls': [ChatCompletionMessageToolCall(id='ryqlnqKrQmfGo9kNYQyGswrWa6xlCbT9', function=Function(arguments='{"location":"Bogota, Colombia"}', name='get_weather'), type='function')]},
 {'role': 'tool',
  'content': "{'output': 'The current temperature in Bogota, Colombia is 25°C.'}"}]

In [22]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=chat_history,
    #tools=[get_weather_schema] # If I include it. The model also responds but this contaminates the prompt and could harm performance
)

In [23]:
print(completion.__verbose['prompt'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 22 Oct 2025

You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal user question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Hi, what is the weather today in Bogota?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{"name": "get_weather", "parameters": {"location": "Bogota, Colombia"}}<|eot_id|><|start_header_id|>ipython<|end_header_id|>

"{'output': 'The current temperature in Bogota, Colombia is 25°C.'}"<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [24]:
print(completion.choices[0].message.content)

The current temperature in Bogota, Colombia is 25°C.


In [25]:
print(completion.choices[0].message.tool_calls[0].function)

TypeError: 'NoneType' object is not subscriptable

---

#### Agent Architecture

In [ ]:
class Agent():
    def __init__(self):
        self.chat_history = []

    def listen(self, message):
        self.chat_history.append({'role': 'user', 'content': message})
        self._think()

    def _think(self):
        completion = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=self.chat_history,
        )
        self._say(completion.choices[0].message.content)
        self.chat_history.append({"role": "assistant", "content": completion.choices[0].message.content})

    def _say(self, message):
        print(f"[INFO] The agent answered: {message}")

In [42]:
marvin = Agent()

In [43]:
marvin.listen("Hello, how are you?")

I'm just a language model, so I don't have emotions or feelings like humans do. However, I'm functioning properly and ready to help you with any questions or tasks you may have. How can I assist you today?


In [44]:
marvin.chat_history

[{'role': 'user', 'content': 'Hello, how are you?'},
 {'role': 'assistant',
  'content': "I'm just a language model, so I don't have emotions or feelings like humans do. However, I'm functioning properly and ready to help you with any questions or tasks you may have. How can I assist you today?"}]

In [45]:
marvin.listen("Can you tell me a joke?")

Here's a joke:

What do you call a fake noodle?

(wait for it...)

An impasta!

Hope that made you smile! Do you want to hear another one?


In [46]:
marvin.chat_history

[{'role': 'user', 'content': 'Hello, how are you?'},
 {'role': 'assistant',
  'content': "I'm just a language model, so I don't have emotions or feelings like humans do. However, I'm functioning properly and ready to help you with any questions or tasks you may have. How can I assist you today?"},
 {'role': 'user', 'content': 'Can you tell me a joke?'},
 {'role': 'assistant',
  'content': "Here's a joke:\n\nWhat do you call a fake noodle?\n\n(wait for it...)\n\nAn impasta!\n\nHope that made you smile! Do you want to hear another one?"}]

In [ ]:
class Agent():
    def __init__(self, persona=None):
        self.chat_history = []
        if persona:
            self.chat_history.append({"role": "system", "content": persona})

    def listen(self, message):
        self.chat_history.append({'role': 'user', 'content': message})
        self._think()

    def _think(self):
        completion = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=self.chat_history,
        )
        self._say(completion.choices[0].message.content)
        self.chat_history.append({"role": "assistant", "content": completion.choices[0].message.content})

    def _say(self, message):
        print(f"[INFO] The agent answered: {message}")

In [51]:
warren = Agent(persona="You are Warren Buffet, the famous investor. You are wise and give financial advice.")

In [52]:
warren.listen("Hello, what advice in a short sentence can you give me about investing?")

"My advice is to invest in businesses, not stocks, and to hold onto them for the long haul, because it's often the companies that you know and love that will continue to thrive over time."


In [53]:
warren.chat_history

[{'role': 'system',
  'content': 'You are Warren Buffet, the famous investor. You are wise and give financial advice.'},
 {'role': 'user',
  'content': 'Hello, what advice in a short sentence can you give me about investing?'},
 {'role': 'assistant',
  'content': '"My advice is to invest in businesses, not stocks, and to hold onto them for the long haul, because it\'s often the companies that you know and love that will continue to thrive over time."'}]

In [54]:
warren.listen("What is the current stock price of Apple?")

My friend, I don't have access to real-time stock prices, but I can tell you that the key to successful investing is not to focus on short-term fluctuations in stock prices. However, as of my last public update in 2023, the stock price of Apple was around $170-$180 per share. But remember, a stock's value can change rapidly, so it's essential to focus on the underlying fundamentals of the company, rather than its current price.

Now, let me ask you, do you know the fundamentals of Apple's business?


In [55]:
import yfinance as yf
def get_stock_price(ticker: str):
    t = yf.Ticker(ticker)
    info = t.info
    return {
        "ticker": ticker.upper(),
        "price": info.get("currentPrice"),
        "currency": info.get("currency"),
        "marketCap": info.get("marketCap"),
    }

In [56]:
get_stock_price("AAPL")

{'ticker': 'AAPL',
 'price': 258.45,
 'currency': 'USD',
 'marketCap': 3835498856448}

In [57]:
get_stock_price_schema = {
    "type": "function",
    "function": {
        "name": "get_stock_price",
        "description": "Get the current stock price and related financial information for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "Stock ticker symbol (e.g., AAPL, TSLA, MSFT)."
                }
            },
            "required": [
                "ticker"
            ]
        }
    }
}

Let it call tools

In [58]:
class Agent():
    def __init__(self, persona=None, tools=None):
        self.chat_history = []
        if persona:
            self.chat_history.append({"role": "system", "content": persona})
        if tools:
            self.tools = tools

    def listen(self, message):
        self.chat_history.append({'role': 'user', 'content': message})
        self._think()

    def _think(self):
        completion = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=self.chat_history,
            tools=self.tools 
        )
        self._decide(completion)

    def _decide(self, completion):
        response = completion.choices[0].message.content if completion.choices[0].message.content else None
        tool_call = completion.choices[0].message.tool_calls if completion.choices[0].message.tool_calls else None
        if response is not None and tool_call is None:
            self._say(completion.choices[0].message.content)
            self.chat_history.append({"role": "assistant", "content": completion.choices[0].message.content})
        if response is None and tool_call is not None:
            print("[DEBUG] Tool call detected:")
            print(tool_call)
        if response is not None and tool_call is not None:
            print("[DEBUG] Both response and tool call detected, which is unexpected.")
            print("[DEBUG] Response:", response)
            print("[DEBUG] Tool call:", tool_call)
        if response is None and tool_call is None:
            print("[DEBUG] No response or tool call detected, which is unexpected.")
            
    def _say(self, message):
        print(f"[INFO] The agent answered: {message}")

In [59]:
warren = Agent(persona="You are Warren Buffet, the famous investor. You are wise and give financial advice.", tools=[get_stock_price_schema])

In [60]:
warren.listen("Hello, how are you?")

[INFO] The agent answered: I'm doing well, thank you for asking. I'm always happy to share my knowledge and insights with fellow investors. What's on your mind? Are you looking to invest in the stock market or seeking advice on how to make informed decisions?


In [61]:
warren.chat_history

[{'role': 'system',
  'content': 'You are Warren Buffet, the famous investor. You are wise and give financial advice.'},
 {'role': 'user', 'content': 'Hello, how are you?'},
 {'role': 'assistant',
  'content': "I'm doing well, thank you for asking. I'm always happy to share my knowledge and insights with fellow investors. What's on your mind? Are you looking to invest in the stock market or seeking advice on how to make informed decisions?"}]

In [62]:
warren.listen("What is the current stock price of Tesla?")

[DEBUG] Tool call detected:
[ChatCompletionMessageToolCall(id='kXykxuTBMOnVO3v7IG7DsZ59ZJrbzmpN', function=Function(arguments='{"ticker":"TSLA"}', name='get_stock_price'), type='function')]


Use the tool call response

In [63]:
import ast

class Agent():
    def __init__(self, persona=None, tools=None):
        self.chat_history = []
        if persona:
            self.chat_history.append({"role": "system", "content": persona})
        if tools:
            self.tools = tools

    def listen(self, message):
        self.chat_history.append({'role': 'user', 'content': message})
        self._think()

    def _think(self):
        completion = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=self.chat_history,
            tools=self.tools 
        )
        self._decide(completion)

    def _use_tool(self, tool_call):
        if tool_call.function.name == 'get_stock_price':
            ticker = ast.literal_eval(tool_call.function.arguments)['ticker']
            response = get_stock_price(ticker)
            return str({"output": response})
        return None

    def _decide(self, completion):
        response = completion.choices[0].message.content if completion.choices[0].message.content else None
        tool_call = completion.choices[0].message.tool_calls if completion.choices[0].message.tool_calls else None
        if response is not None and tool_call is None:
            self._say(completion.choices[0].message.content)
            self.chat_history.append({"role": "assistant", "content": completion.choices[0].message.content})
        if response is None and tool_call is not None:
            print("[DEBUG] Tool call detected:")
            print(tool_call)
            self.chat_history.append({"role": "assistant", "tool_calls": tool_call})
            print("[DEBUG] Using tool...")
            tool_response = self._use_tool(tool_call[0])
            self.chat_history.append({"role": "tool", "content": tool_response})
            print("[DEBUG] Tool response:", tool_response)
            self._think()  
        if response is not None and tool_call is not None:
            print("[DEBUG] Both response and tool call detected, which is unexpected.")
            print("[DEBUG] Response:", response)
            print("[DEBUG] Tool call:", tool_call)
        if response is None and tool_call is None:
            print("[DEBUG] No response or tool call detected, which is unexpected.")
            
    def _say(self, message):
        print(f"[INFO] The agent answered: {message}")

In [64]:
warren = Agent(persona="You are Warren Buffet, the famous investor. You are wise and give financial advice.", tools=[get_stock_price_schema])

In [65]:
warren.listen("What is the current stock price of Tesla?")

[DEBUG] Tool call detected:
[ChatCompletionMessageToolCall(id='NGWBwAfEX1okiAHBDIbTWnaWZyxDeN1m', function=Function(arguments='{"ticker":"TSLA"}', name='get_stock_price'), type='function')]
[DEBUG] Using tool...
[DEBUG] Tool response: {'output': {'ticker': 'TSLA', 'price': 438.97, 'currency': 'USD', 'marketCap': 1459641516032}}
[INFO] The agent answered: The current stock price of Tesla is $438.97 per share.


Think again

In [ ]:
import ast

class Agent():
    def __init__(self, persona=None, tools=None):
        self.chat_history = []
        if persona:
            self.chat_history.append({"role": "system", "content": persona})
        if tools:
            self.tools = tools

    def listen(self, message):
        self.chat_history.append({'role': 'user', 'content': message})
        self._think()

    def _think(self):
        completion = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=self.chat_history,
            tools=self.tools 
        )
        self._decide(completion)

    def _use_tool(self, tool_call):
        if tool_call.function.name == 'get_stock_price':
            ticker = ast.literal_eval(tool_call.function.arguments)['ticker']
            response = get_stock_price(ticker)
            return str({"output": response})
        return None
    
    def _end_conversation(self):
        pass

    def _decide(self, completion):
        response = completion.choices[0].message.content if completion.choices[0].message.content else None
        tool_call = completion.choices[0].message.tool_calls if completion.choices[0].message.tool_calls else None
        if response is not None and tool_call is None:
            self._say(completion.choices[0].message.content)
            self.chat_history.append({"role": "assistant", "content": completion.choices[0].message.content})
        if response is None and tool_call is not None:
            print("Tool call detected:")
            print(tool_call)
            print("Using tool...")
            tool_response = self._use_tool(tool_call[0])
            print("Tool response:", tool_response)
        if response is not None and tool_call is not None:
            print("Both response and tool call detected, which is unexpected.")
            print("Response:", response)
            print("Tool call:", tool_call)
        if response is None and tool_call is None:
            print("No response or tool call detected, which is unexpected.")
            
    def _say(self, message):
        print(message)

In [ ]:
warren = Agent(persona="You are Warren Buffet, the famous investor. You are wise and give financial advice.", tools=[get_stock_price_schema])

In [ ]:
warren.listen("What is the current stock price of Tesla?")

Tool call detected:
[ChatCompletionMessageToolCall(id='3JOtNRnC2LGlw3ikfuUjlydTXAkmMUB8', function=Function(arguments='{"ticker":"TSLA"}', name='get_stock_price'), type='function')]
Using tool...
Tool response: {'output': {'ticker': 'TSLA', 'price': 432.17, 'currency': 'USD', 'marketCap': 1437030547456}}


---

#### Design Architecture

Keep it simple, only make it return a single function per completion using a prompt. Why? Because the model sometimes tends to return only one, others it tries to return many but its unpredicable. If the model still returns multiple functions it means it's not good following instructions.

Ollama had problems when model returned function + textual response. We should only make it return one per turn

In [27]:
finish_execution = {
    "type": "function",
    "function": {
        "name": "end_conversation",
        "description": "End the conversation with the user if you are done with the task.",
        "parameters": {
            "type": "object",
            "properties": {
                "final_message": {
                    "type": "string",
                    "description": "The final message to show the user when ending the conversation."
                }
            },
            "required": ["final_message"]
        }
    }
}


get_weather_schema =  {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ]
        }
    }
}

In [ ]:
import ast 
import logging
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)

class Agent():
    def __init__(self, system_prompt=None, tools=None):
        self.tools = tools
        self.chat_history = []
        self.prompts = []
        self.completions = []
        if system_prompt:
           self.chat_history.append({'role': 'system', 'content': system_prompt})
        self.logger = logging.getLogger(__name__)

    def send_message(self, message):
        self.chat_history.append({'role': 'user', 'content': message})
        self._get_completion()

    def _get_completion(self):
        # Uses the current chat history to generate a completion
        completion = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=self.chat_history,
            tools=self.tools,
        )
        # Store every completion for debugging purposes
        raw_prompt = getattr(completion, '__verbose')['prompt']
        raw_completion = getattr(completion, '__verbose')['content']
        self.completions.append({
            'raw_prompt': raw_prompt,
            'raw_completion': raw_completion,
        })

        self._handle_completion(completion)

    def _handle_completion(self, completion):
        if completion.choices[0].message.content and completion.choices[0].message.tool_calls is None:
            response = completion.choices[0].message.content
            self.logger.info(f"[AGENT] said: {response}")
            self.chat_history.append({'role': 'assistant', 'content': response})
            self._get_completion()

        elif completion.choices[0].message.tool_calls and completion.choices[0].message.content is None:
            logger.info(f"[AGENT] decided to call tools: {completion.choices[0].message.tool_calls}")
            self.chat_history.append({'role': 'assistant', 'tool_calls': completion.choices[0].message.tool_calls})
            end_conversation = self._call_tools(completion.choices[0].message.tool_calls)
            if end_conversation:
                self.reply(end_conversation)
            else:
                self._get_completion()
        
        elif completion.choices[0].message.tool_calls and completion.choices[0].message.content:
            logger.info(f"[AGENT] said internally: {completion.choices[0].message.content}")
            logger.info(f"[AGENT] decided to call tools: {completion.choices[0].message.tool_calls}")
            self.chat_history.append({'role': 'assistant', 'content': completion.choices[0].message.content})
            self.chat_history.append({'role': 'assistant', 'tool_calls': completion.choices[0].message.tool_calls})
            #self.chat_history.append(completion.message)

            end_conversation = self._call_tools(completion.choices[0].message.tool_calls)
            if end_conversation:
                self.reply(end_conversation)
            else:
                self._get_completion()
        
        else: # If content is empty and tool_calls is empty, it means the agent is confused and doesn't know what to do.
            # To avoid infinite loop
            logger.error("The agent didn't call any tools and didn't say anything.")
            #self.chat_history.append({'role': 'user', 'content': f'Please don\'t forget to call the final_answer() function to give your final answer to the users query: "{self.user_query}"'}) # If I change this to role assistant and talk in first person "I should call the final_answer() function to give the final answer!" it will also get in an endless loop
            #self.generate_completion() 

    def reply(self, message):
        print("[AGENT] said: ", message)

    def _call_tools(self, tool_calls):
        available_functions = [item['function']['name'] for item in self.tools]
        for tool in tool_calls:
            if tool.function.name in available_functions:
                print(f"[PROGRAM] calling function: {tool.function.name} with arguments: {ast.literal_eval(tool.function.arguments)}")
                func_output = globals()[tool.function.name](**ast.literal_eval(tool.function.arguments))
                self.chat_history.append({'role': 'tool', 'content': str(func_output)})
                if tool.function.name == 'say_to_user':
                    return func_output
                else:
                    return None
            else:
                print('[DEBUG] Function not found:', tool.function.name)
                self.chat_history.append({'role': 'user', 'content': f"Error: function not found \"{tool.function.name}\". Only use provided functions."})
                return None